In [ ]:
# faceMask hDDM face ratings analysis script: mask analyses of false negatives (failing to ID correct emotion)
# 12/07/20

# note: to be run within python 3.5 environment named "hddm" (from console: 'source activate hddm')
# confirm jupyter notebook is launching from "hddm" environment (see top-right: 'conda env:hddm')

# set up
import numpy as np
print(np.__version__) # should be 1.11.3

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from patsy import dmatrix

import hddm
print(hddm.__version__) # should be 0.6.0

In [ ]:
# load hDDM-processed data from CSV file into a NumPy structured array
data = hddm.load_csv('faceMask2_hddm_proc.csv')
                     
# flip 'no' response RTs to be negative
data = hddm.utils.flip_errors(data)

# check dataset
data.head(10)

In [ ]:
# plot RTs for each participant
fig = plt.figure()
ax = fig.add_subplot(111, xlabel='RT', ylabel='count', title='RT distributions')
for i, subj_data in data.groupby('subj_idx'):
    subj_data.rt.hist(bins=20, histtype='step', ax=ax)

In [ ]:
# subset dataset by emotion rating type (happy) and facial expression (sad)
hs_data = data.loc[(data['emotionRating'] == 'happy') & (data['expression'] == 'sad')]
hs_data.head(10)

In [ ]:
# drop any subjects missing columns in design matrix (otherwise produces error)
hs_data = hs_data[(hs_data.subj_idx != 'dqph4ch0mkofejw') & (hs_data.subj_idx != 'hdn2l3df216jykn') & (hs_data.subj_idx != 'j2qezlblp8v0dh6') & (hs_data.subj_idx != 'me8doxrmo9vj9dx')]

In [ ]:
# define model
m_hs = hddm.HDDMRegressor(hs_data, "v ~ C(mask)", bias=True, p_outlier=0.05)

In [ ]:
# run model
m_hs.sample(5000, burn=200, dbname='traces.db', db='pickle')
m_hs.print_stats()
m_hs.plot_posteriors()

In [ ]:
# save model stats
hs_stats = m_hs.gen_stats()
print(hs_stats)
hs_stats.to_csv('hs_5000/faceMask_hddm_drift_maskFalsePositives_hs_5000.csv', index=True)

# save model posterior plots
m_hs.plot_posteriors(path='hs_5000/_posteriors', save=True)

In [ ]:
# plot model posteriors by mask
hs_v_none, hs_v_lower, hs_v_upper = m_hs.nodes_db.node[["v_Intercept", "v_C(mask)[T.lower]", "v_C(mask)[T.upper]"]]
hddm.analyze.plot_posterior_nodes([hs_v_none, hs_v_lower, hs_v_upper])
plt.savefig('hs_5000/faceMask_hddm_drift_maskFalsePositives_hs_5000_v_mask.pdf')

In [ ]:
## estimate probabilities that mask coefficient posteriors differ from 0
# note that comparison condition coefficients (i.e. lower/upper) are *relative to* baseline condition (i.e. no mask)
# for lower/upper, =0 means no change from baseline, <0 means less than baseline, and >0 means greater than baseline
# for no mask, =0 means null drift, <0 means negative drift, and >0 means positive drift
print("P(hs_v_none < 0) = ", (hs_v_none.trace() < 0).mean())
print("P(hs_v_lower < 0) = ", (hs_v_lower.trace() < 0).mean())
print("P(hs_v_upper > 0) = ", (hs_v_upper.trace() > 0).mean())

# estimate probability that lower and upper mask coefficient posteriors differ
print("P(hs_v_lower < hs_v_upper) = ", (hs_v_lower.trace() < hs_v_upper.trace()).mean())